<a href="https://colab.research.google.com/github/kleofas97/PracaMagisterka/blob/main/Gated_Trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggleDownloader
!pip install -q kaggle
from kaggleDownloader import get_dataset
from google.colab import files


In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
files.upload()

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adammika","key":"e7d62e5b1bd17724efe523a33f035198"}'}

In [ ]:
get_dataset("kaggle datasets download -d adammika/iam-lines-in-folder")


Download completed..
Unzipping the zip files

Zip Files unzipped

 Directory contains the following files :  ['.config', 'img', 'iam-lines-in-folder.zip', 'kaggle.json', 'sample_data']

Remove zip files ? (yes/no) :no

 Directory contains the following files :  ['.config', 'img', 'iam-lines-in-folder.zip', 'kaggle.json', 'sample_data']


In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import random
import cv2
import argparse
import json
import pickle
import editdistance
from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
import keras.backend as K

In [ ]:
class Sample:
    "sample from the dataset"

    def __init__(self, gtText, filePath):
        self.gtText = gtText
        self.filePath = filePath


class Batch:
    "batch containing images and ground truth texts"

    def __init__(self, gtTexts, imgs):
        self.imgs = np.stack(imgs, axis=0)
        self.gtTexts = gtTexts


class DataLoaderIAM:
    "loads data which corresponds to IAM format, see: http://www.fki.inf.unibe.ch/databases/iam-handwriting-database"

    def __init__(self, data_dir, batchSize, imgSize, maxTextLen, fast=True):
        "loader for dataset at given location, preprocess images and text according to parameters"

        assert os.path.isdir(data_dir)

        self.fast = fast
        # if fast:
        #     self.env = lmdb.open(str(data_dir / 'lmdb'), readonly=True)

        self.dataAugmentation = False
        self.currIdx = 0
        self.batchSize = batchSize
        self.imgSize = imgSize
        self.samples = []
        print(os.path.join(data_dir, 'lines.txt'))
        f = open(os.path.join(data_dir, 'lines.txt'))
        chars = set()
        bad_samples_reference = ['a01-117-05-02',
                                 'r06-022-03-05']  # known broken images in IAM dataset
        for line in f:
            # ignore comment line
            if not line or line[0] == '#':
                continue

            lineSplit = line.strip().split(' ')
            assert len(lineSplit) >= 9

            # filename: part1-part2-part3 --> part1/part1-part2/part1-part2-part3.png
            fileNameSplit = lineSplit[0].split('-')
            fileName = data_dir + "/" + 'img' + "/" + fileNameSplit[
                0] + "/" + f'{fileNameSplit[0]}-{fileNameSplit[1]}' + "/" + lineSplit[0] + '.png'

            if lineSplit[0] in bad_samples_reference:
                print('Ignoring known broken image:', fileName)
                continue

            # GT text are columns starting at 9
            gtText = self.truncateLabel(' '.join(lineSplit[8:]), maxTextLen)
            chars = chars.union(set(list(gtText)))

            # put sample into list
            self.samples.append(Sample(gtText, fileName))

        # split into training and validation set: 95% - 5%
        splitIdx = int(0.95 * len(self.samples))
        self.trainSamples = self.samples[:splitIdx]
        self.validationSamples = self.samples[splitIdx:]

        # put words into lists
        self.trainWords = [x.gtText for x in self.trainSamples]
        self.validationWords = [x.gtText for x in self.validationSamples]

        # start with train set
        self.trainSet()

        # list of all chars in dataset
        self.charList = sorted(list(chars))

    def truncateLabel(self, text, maxTextLen):
        # ctc_loss can't compute loss if it cannot find a mapping between text label and input
        # labels. Repeat letters cost double because of the blank symbol needing to be inserted.
        # If a too-long label is provided, ctc_loss returns an infinite gradient
        cost = 0
        for i in range(len(text)):
            if i != 0 and text[i] == text[i - 1]:
                cost += 2
            else:
                cost += 1
            if cost > maxTextLen:
                return text[:i]
        return text

    def trainSet(self):
        "switch to randomly chosen subset of training set"
        self.dataAugmentation = False
        self.currIdx = 0
        random.shuffle(self.trainSamples)
        self.samples = self.trainSamples
        self.currSet = 'train'

    def validationSet(self):
        "switch to validation set"
        self.dataAugmentation = False
        self.currIdx = 0
        self.samples = self.validationSamples
        self.currSet = 'val'

    def getIteratorInfo(self):
        "current batch index and overall number of batches"
        if self.currSet == 'train':
            numBatches = int(
                np.floor(len(self.samples) / self.batchSize))  # train set: only full-sized batches
        else:
            numBatches = int(np.ceil(
                len(self.samples) / self.batchSize))  # val set: allow last batch to be smaller
        currBatch = self.currIdx // self.batchSize + 1
        return currBatch, numBatches

    def hasNext(self):
        "iterator"
        if self.currSet == 'train':
            return self.currIdx + self.batchSize <= len(
                self.samples)  # train set: only full-sized batches
        else:
            return self.currIdx < len(self.samples)  # val set: allow last batch to be smaller

    def getNext(self):
        "iterator"
        batchRange = range(self.currIdx, min(self.currIdx + self.batchSize, len(self.samples)))
        gtTexts = [self.samples[i].gtText for i in batchRange]

        imgs = []
        for i in batchRange:
            # if self.fast:
            #     with self.env.begin() as txn:
            #         basename = Path(self.samples[i].filePath).basename()
            #         data = txn.get(basename.encode("ascii"))
            #         img = pickle.loads(data)
            # else:
            img = cv2.imread(self.samples[i].filePath, cv2.IMREAD_GRAYSCALE)

            imgs.append(preprocess(img, self.imgSize, self.dataAugmentation))
        self.currIdx += self.batchSize
        return Batch(gtTexts, imgs)



In [ ]:
def preprocess(img, imgSize, dataAugmentation=False):
    "put img into target img of size imgSize, transpose for TF and normalize gray-values"

    # there are damaged files in IAM dataset - just use black image instead
    if img is None:
        img = np.zeros(imgSize[::-1])

    # data augmentation
    img = img.astype(np.float)
    if dataAugmentation:
        # photometric data augmentation
        if random.random() < 0.25:
            rand_odd = lambda: random.randint(1, 3) * 2 + 1
            img = cv2.GaussianBlur(img, (rand_odd(), rand_odd()), 0)
        if random.random() < 0.25:
            img = cv2.dilate(img, np.ones((3, 3)))
        if random.random() < 0.25:
            img = cv2.erode(img, np.ones((3, 3)))
        if random.random() < 0.5:
            img = img * (0.25 + random.random() * 0.75)
        if random.random() < 0.25:
            img = np.clip(img + (np.random.random(img.shape) - 0.5) * random.randint(1, 50), 0, 255)
        if random.random() < 0.1:
            img = 255 - img

        # geometric data augmentation
        wt, ht = imgSize
        h, w = img.shape
        f = min(wt / w, ht / h)
        fx = f * np.random.uniform(0.75, 1.25)
        fy = f * np.random.uniform(0.75, 1.25)

        # random position around center
        txc = (wt - w * fx) / 2
        tyc = (ht - h * fy) / 2
        freedom_x = max((wt - fx * w) / 2, 0) + wt / 10
        freedom_y = max((ht - fy * h) / 2, 0) + ht / 10
        tx = txc + np.random.uniform(-freedom_x, freedom_x)
        ty = tyc + np.random.uniform(-freedom_y, freedom_y)

        # map image into target image
        M = np.float32([[fx, 0, tx], [0, fy, ty]])
        target = np.ones(imgSize[::-1]) * 255 / 2
        img = cv2.warpAffine(img, M, dsize=imgSize, dst=target, borderMode=cv2.BORDER_TRANSPARENT)

    # no data augmentation
    else:
        # center image
        wt, ht = imgSize
        h, w = img.shape
        f = min(wt / w, ht / h)
        tx = (wt - w * f) / 2
        ty = (ht - h * f) / 2

        # map image into target image
        M = np.float32([[f, 0, tx], [0, f, ty]])
        target = np.ones(imgSize[::-1]) * 255 / 2
        img = cv2.warpAffine(img, M, dsize=imgSize, dst=target, borderMode=cv2.BORDER_TRANSPARENT)

    # transpose for TF
    img = cv2.transpose(img)

    # convert to range [-1, 1]
    img = img / 255 - 0.5
    return img


# if __name__ == '__main__':
#     import matplotlib.pyplot as plt

#     img = cv2.imread('../data/test.png', cv2.IMREAD_GRAYSCALE)
#     img_aug = preprocess(img, (128, 32), True)
#     plt.subplot(121)
#     plt.imshow(img, cmap='gray')
#     plt.subplot(122)
#     plt.imshow(cv2.transpose(img_aug), cmap='gray')
#     plt.show()


In [ ]:
# Disable eager mode
tf.compat.v1.disable_eager_execution()


class DecoderType:
    BestPath = 0
    BeamSearch = 1
    WordBeamSearch = 2


class Model:
    "minimalistic TF model for HTR"

    # model constants
    imgSize = (512, 48)
    maxTextLen = 128

    def __init__(self, charList, decoderType=DecoderType.BestPath, mustRestore=False, dump=False):
        "init model: add CNN, RNN and CTC and initialize TF"
        self.dump = dump
        self.charList = charList
        self.decoderType = decoderType
        self.mustRestore = mustRestore
        self.snapID = 0

        # Whether to use normalization over a batch or a population
        self.is_train = tf.compat.v1.placeholder(tf.bool, name='is_train')

        # input image batch
        self.inputImgs = tf.compat.v1.placeholder(tf.float32, shape=(None, Model.imgSize[0], Model.imgSize[1]))

        

        # setup CNN, RNN and CTC
        self.setupCNN()
        self.setupRNN()
        self.setupCTC()

        # setup optimizer to train NN
        self.batchesTrained = 0
        self.update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(self.update_ops):
            self.optimizer = tf.compat.v1.train.RMSPropOptimizer(learning_rate=0.001).minimize(self.loss)

        # initialize TF
        (self.sess, self.saver) = self.setupTF()

    def setupCNN(self):
        "create CNN layers and return output of these layers"

        cnnIn4d = tf.expand_dims(input=self.inputImgs, axis=3)


        kernel = tf.Variable(tf.random.truncated_normal([3,3,1,8],stddev=0.1))
        conv = tf.nn.conv2d(input=cnnIn4d, filters=kernel, padding='SAME', strides=(1, 1, 1, 1))
        tanh = tf.nn.tanh(conv)
        kernel = tf.Variable(tf.random.truncated_normal([2,4,8,16],stddev=0.1))
        conv = tf.nn.conv2d(input=tanh, filters=kernel, padding='SAME', strides=(1, 1, 1, 1))
        tanh = tf.nn.tanh(conv)
        kernel = tf.Variable(tf.random.truncated_normal([3, 3,16, 16], stddev=0.1))
        conv_gate = tf.nn.conv2d(input=tanh, filters=kernel, padding='SAME', strides=(1, 1, 1,1))
        sigmoid = tf.nn.sigmoid(conv_gate)
        output = tf.multiply(sigmoid, tanh)
        kernel = tf.Variable(tf.random.truncated_normal([3, 3, 16, 32], stddev=0.1))
        conv = tf.nn.conv2d(input=output, filters=kernel, padding='SAME', strides=(1, 1, 1, 1))
        tanh = tf.nn.tanh(conv)
        kernel = tf.Variable(tf.random.truncated_normal([3, 3, 32,32], stddev=0.1))
        conv_gate = tf.nn.conv2d(input=tanh, filters=kernel, padding='SAME', strides=(1, 1, 1,1))
        sigmoid = tf.nn.sigmoid(conv_gate)
        output = tf.multiply(sigmoid, tanh)
        kernel = tf.Variable(tf.random.truncated_normal([2, 4, 32, 64], stddev=0.1))
        conv = tf.nn.conv2d(input=output, filters=kernel, padding='SAME', strides=(1, 1, 1, 1))
        tanh = tf.nn.tanh(conv)
        kernel = tf.Variable(tf.random.truncated_normal([3, 3,64, 64], stddev=0.1))
        conv_gate = tf.nn.conv2d(input=tanh, filters=kernel, padding='SAME', strides=(1, 1, 1,1))
        sigmoid = tf.nn.sigmoid(conv_gate)
        output = tf.multiply(sigmoid, tanh)
        kernel = tf.Variable(tf.random.truncated_normal([3, 3, 64, 128], stddev=0.1))
        conv = tf.nn.conv2d(input=output, filters=kernel, padding='SAME', strides=(1, 1, 1, 1))
        tanh = tf.nn.tanh(conv)

        pool = tf.nn.max_pool2d(input=tanh, ksize=(1, 1, Model.imgSize[1], 1),
                                strides=(1, 1, 1, 1), padding='VALID')

        self.cnnOut4d = pool

    def setupRNN(self):
        "create RNN layers and return output of these layers"

        numHidden = 128
        rnnIn3d = tf.squeeze(self.cnnOut4d, axis=[2])
        blstm_1 = Bidirectional(LSTM(numHidden, return_sequences=True))(rnnIn3d)
        dense1 = Dense(128, activation = 'tanh')(blstm_1)
        blstm_2 = Bidirectional(LSTM(numHidden, return_sequences=True))(dense1)
        dense1 = Dense(128, activation = 'tanh')(blstm_2)
    
        concat = tf.expand_dims(dense1,2)
        print("concat: {}".format(concat.shape))

        # project output to chars (including blank): BxTx1x2H -> BxTx1xC -> BxTxC
        kernel = tf.Variable(
            tf.random.truncated_normal([1, 1, numHidden, len(self.charList) + 1], stddev=0.1))
        self.rnnOut3d = tf.squeeze(
            tf.nn.atrous_conv2d(value=concat, filters=kernel, rate=1, padding='SAME'), axis=[2])

    def setupCTC(self):
        "create CTC loss and decoder and return them"
        # BxTxC -> TxBxC
        self.ctcIn3dTBC = tf.transpose(a=self.rnnOut3d, perm=[1, 0, 2])
        # ground truth text as sparse tensor
        self.gtTexts = tf.SparseTensor(tf.compat.v1.placeholder(tf.int64, shape=[None, 2]),
                                       tf.compat.v1.placeholder(tf.int32, [None]),
                                       tf.compat.v1.placeholder(tf.int64, [2]))

        # calc loss for batch
        self.seqLen = tf.compat.v1.placeholder(tf.int32, [None])
        self.loss = tf.reduce_mean(input_tensor=tf.compat.v1.nn.ctc_loss(labels=self.gtTexts, inputs=self.ctcIn3dTBC,
                                                                         sequence_length=self.seqLen,
                                                                         ctc_merge_repeated=True))

        # calc loss for each element to compute label probability
        self.savedCtcInput = tf.compat.v1.placeholder(tf.float32,
                                                      shape=[Model.maxTextLen, None, len(self.charList) + 1])
        self.lossPerElement = tf.compat.v1.nn.ctc_loss(labels=self.gtTexts, inputs=self.savedCtcInput,
                                                       sequence_length=self.seqLen, ctc_merge_repeated=True)

        # best path decoding or beam search decoding
        if self.decoderType == DecoderType.BestPath:
            self.decoder = tf.nn.ctc_greedy_decoder(inputs=self.ctcIn3dTBC, sequence_length=self.seqLen)
        elif self.decoderType == DecoderType.BeamSearch:
            self.decoder = tf.nn.ctc_beam_search_decoder(inputs=self.ctcIn3dTBC, sequence_length=self.seqLen,
                                                         beam_width=50)
        # word beam search decoding (see https://github.com/githubharald/CTCWordBeamSearch)
        elif self.decoderType == DecoderType.WordBeamSearch:
            # prepare information about language (dictionary, characters in dataset, characters forming words)
            chars = str().join(self.charList)
            wordChars = open('/content/wordCharList.txt').read().splitlines()[0]
            corpus = open('/content/corpus.txt').read()

            # decode using the "Words" mode of word beam search
            from word_beam_search import WordBeamSearch
            self.decoder = WordBeamSearch(50, 'Words', 0.0, corpus.encode('utf8'), chars.encode('utf8'),
                                          wordChars.encode('utf8'))

            # the input to the decoder must have softmax already applied
            self.wbsInput = tf.nn.softmax(self.ctcIn3dTBC, axis=2)

    def setupTF(self):
        "initialize TF"
        print('Python: ' + sys.version)
        print('Tensorflow: ' + tf.__version__)

        sess = tf.compat.v1.Session()  # TF session

        saver = tf.compat.v1.train.Saver(max_to_keep=1)  # saver saves model to file
        modelDir = r'/content/model'
        latestSnapshot = tf.train.latest_checkpoint(modelDir)  # is there a saved model?

        # if model must be restored (for inference), there must be a snapshot
        if self.mustRestore and not latestSnapshot:
            raise Exception('No saved model found in: ' + modelDir)

        # load saved model if available
        if latestSnapshot:
            print('Init with stored values from ' + latestSnapshot)
            saver.restore(sess, latestSnapshot)
        else:
            print('Init with new values')
            sess.run(tf.compat.v1.global_variables_initializer())

        return (sess, saver)

    def toSparse(self, texts):
        "put ground truth texts into sparse tensor for ctc_loss"
        indices = []
        values = []
        shape = [len(texts), 0]  # last entry must be max(labelList[i])

        # go over all texts
        for (batchElement, text) in enumerate(texts):
            # convert to string of label (i.e. class-ids)
            labelStr = [self.charList.index(c) for c in text]
            # sparse tensor must have size of max. label-string
            if len(labelStr) > shape[1]:
                shape[1] = len(labelStr)
            # put each label into sparse tensor
            for (i, label) in enumerate(labelStr):
                indices.append([batchElement, i])
                values.append(label)

        return (indices, values, shape)

    def decoderOutputToText(self, ctcOutput, batchSize):
        "extract texts from output of CTC decoder"

        # word beam search: already contains label strings
        if self.decoderType == DecoderType.WordBeamSearch:
            labelStrs = ctcOutput

        # TF decoders: label strings are contained in sparse tensor
        else:
            # ctc returns tuple, first element is SparseTensor
            decoded = ctcOutput[0][0]

            # contains string of labels for each batch element
            labelStrs = [[] for _ in range(batchSize)]

            # go over all indices and save mapping: batch -> values
            for (idx, idx2d) in enumerate(decoded.indices):
                label = decoded.values[idx]
                batchElement = idx2d[0]  # index according to [b,t]
                labelStrs[batchElement].append(label)

        # map labels to chars for all batch elements
        return [str().join([self.charList[c] for c in labelStr]) for labelStr in labelStrs]

    def trainBatch(self, batch):
        "feed a batch into the NN to train it"
        numBatchElements = len(batch.imgs)
        sparse = self.toSparse(batch.gtTexts)
        evalList = [self.optimizer, self.loss]
        feedDict = {self.inputImgs: batch.imgs, self.gtTexts: sparse,
                    self.seqLen: [Model.maxTextLen] * numBatchElements, self.is_train: True}
        _, lossVal = self.sess.run(evalList, feedDict)
        self.batchesTrained += 1
        return lossVal

    def dumpNNOutput(self, rnnOutput):
        "dump the output of the NN to CSV file(s)"
        dumpDir = '../dump/'
        if not os.path.isdir(dumpDir):
            os.mkdir(dumpDir)

        # iterate over all batch elements and create a CSV file for each one
        maxT, maxB, maxC = rnnOutput.shape
        for b in range(maxB):
            csv = ''
            for t in range(maxT):
                for c in range(maxC):
                    csv += str(rnnOutput[t, b, c]) + ';'
                csv += '\n'
            fn = dumpDir + 'rnnOutput_' + str(b) + '.csv'
            print('Write dump of NN to file: ' + fn)
            with open(fn, 'w') as f:
                f.write(csv)

    def inferBatch(self, batch, calcProbability=False, probabilityOfGT=False):
        "feed a batch into the NN to recognize the texts"

        # decode, optionally save RNN output
        numBatchElements = len(batch.imgs)

        # put tensors to be evaluated into list
        evalList = []

        if self.decoderType == DecoderType.WordBeamSearch:
            evalList.append(self.wbsInput)
        else:
            evalList.append(self.decoder)

        if self.dump or calcProbability:
            evalList.append(self.ctcIn3dTBC)

        # dict containing all tensor fed into the model
        feedDict = {self.inputImgs: batch.imgs, self.seqLen: [Model.maxTextLen] * numBatchElements,
                    self.is_train: False}

        # evaluate model
        evalRes = self.sess.run(evalList, feedDict)

        # TF decoders: decoding already done in TF graph
        if self.decoderType != DecoderType.WordBeamSearch:
            decoded = evalRes[0]
        # word beam search decoder: decoding is done in C++ function compute()
        else:
            decoded = self.decoder.compute(evalRes[0])

        # map labels (numbers) to character string
        texts = self.decoderOutputToText(decoded, numBatchElements)

        # feed RNN output and recognized text into CTC loss to compute labeling probability
        probs = None
        if calcProbability:
            sparse = self.toSparse(batch.gtTexts) if probabilityOfGT else self.toSparse(texts)
            ctcInput = evalRes[1]
            evalList = self.lossPerElement
            feedDict = {self.savedCtcInput: ctcInput, self.gtTexts: sparse,
                        self.seqLen: [Model.maxTextLen] * numBatchElements, self.is_train: False}
            lossVals = self.sess.run(evalList, feedDict)
            probs = np.exp(-lossVals)

        # dump the output of the NN to CSV file(s)
        if self.dump:
            self.dumpNNOutput(evalRes[1])

        return texts, probs

    def save(self):
        "save model to file"
        self.snapID += 1
        self.saver.save(self.sess, '/content/model/snapshot', global_step=self.snapID)


In [ ]:



class FilePaths:
    "filenames and paths to data"
    fnCharList = r'/content/wordCharList.txt'
    fnSummary = r'/content/summary.json'
    fnInfer = r'/content/test.png'
    fnCorpus = r'/content/corpus.txt'


def write_summary(charErrorRates, wordAccuracies):
    with open(FilePaths.fnSummary, 'w') as f:
        json.dump({'charErrorRates': charErrorRates, 'wordAccuracies': wordAccuracies}, f)


def train(model, loader):
    "train NN"
    epoch = 0  # number of training epochs since start
    summaryCharErrorRates = []
    summaryWordAccuracies = []
    bestCharErrorRate = float('inf')  # best valdiation character error rate
    noImprovementSince = 0  # number of epochs no improvement of character error rate occured
    earlyStopping = 5  # stop training after this number of epochs without improvement
    while True:
        epoch += 1
        print('Epoch:', epoch)

        # train
        print('Train NN')
        loader.trainSet()
        while loader.hasNext():
            iterInfo = loader.getIteratorInfo()
            batch = loader.getNext()
            loss = model.trainBatch(batch)
            print(f'Epoch: {epoch} Batch: {iterInfo[0]}/{iterInfo[1]} Loss: {loss}')

        # validate
        charErrorRate, wordAccuracy = validate(model, loader)

        # write summary
        summaryCharErrorRates.append(charErrorRate)
        summaryWordAccuracies.append(wordAccuracy)
        write_summary(summaryCharErrorRates, summaryWordAccuracies)

        # if best validation accuracy so far, save model parameters
        if charErrorRate < bestCharErrorRate:
            print('Character error rate improved, save model')
            bestCharErrorRate = charErrorRate
            noImprovementSince = 0
            model.save()
        else:
            print(f'Character error rate not improved, best so far: {charErrorRate * 100.0}%')
            noImprovementSince += 1

        # stop training if no more improvement in the last x epochs
        if noImprovementSince >= earlyStopping:
            print(f'No more improvement since {earlyStopping} epochs. Training stopped.')
            break

        if epoch>5:
          break


def validate(model, loader):
    "validate NN"
    print('Validate NN')
    loader.validationSet()
    numCharErr = 0
    numCharTotal = 0
    numWordOK = 0
    numWordTotal = 0
    while loader.hasNext():
        iterInfo = loader.getIteratorInfo()
        print(f'Batch: {iterInfo[0]} / {iterInfo[1]}')
        batch = loader.getNext()
        (recognized, _) = model.inferBatch(batch)

        print('Ground truth -> Recognized')
        for i in range(len(recognized)):
            numWordOK += 1 if batch.gtTexts[i] == recognized[i] else 0
            numWordTotal += 1
            dist = editdistance.eval(recognized[i], batch.gtTexts[i])
            numCharErr += dist
            numCharTotal += len(batch.gtTexts[i])
            print('[OK]' if dist == 0 else '[ERR:%d]' % dist, '"' + batch.gtTexts[i] + '"', '->',
                  '"' + recognized[i] + '"')

    # print validation result
    charErrorRate = numCharErr / numCharTotal
    wordAccuracy = numWordOK / numWordTotal
    print(
        f'Character error rate: {charErrorRate * 100.0}%. Word accuracy: {wordAccuracy * 100.0}%.')
    return charErrorRate, wordAccuracy


def infer(model, fnImg):
    "recognize text in image provided by file path"
    img = preprocess(cv2.imread(fnImg, cv2.IMREAD_GRAYSCALE), Model.imgSize)
    batch = Batch(None, [img])
    (recognized, probability) = model.inferBatch(batch, True)
    print(f'Recognized: "{recognized[0]}"')
    print(f'Probability: {probability[0]}')


def infer_matlab(imgs):
    "recognize text in image provided by file path"
    img_ready = []
    decoderType = DecoderType.BestPath
    model = Model(open(FilePaths.fnCharList).read(), decoderType, mustRestore=True)
    for img in imgs:
        img = preprocess(img, Model.imgSize)
        img_ready.append(img)
    batch = Batch(None, img_ready)
    (recognized, probability) = model.inferBatch(batch, True)
    return recognized, probability


def main():
    "main function"
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--train', help='train the NN', action='store_true', default=True)
    # parser.add_argument('--validate', help='validate the NN', action='store_true')
    # parser.add_argument('--decoder', choices=['bestpath', 'beamsearch', 'wordbeamsearch'],
    #                     default='bestpath',
    #                     help='CTC decoder')
    # parser.add_argument('--batch_size', help='batch size', type=int, default=16)
    # parser.add_argument('--data_dir', help='directory containing IAM dataset',
    #                     required=False, default="/content/")
    # parser.add_argument('--fast', help='use lmdb to load images', action='store_true',
    #                     default=False)
    # parser.add_argument('--dump', help='dump output of NN to CSV file(s)', action='store_true')
    # args = parser.parse_args()
    train_b = True
    validate = False
    data_dir = "/content/"
    batch_size = 128
    imgSize = (512, 48)
    maxTextLen= 128
    fast = False
    decoder = 'bestpath'
    # set chosen CTC decoder
    if decoder == 'bestpath':
        decoderType = DecoderType.BestPath
    elif decoder == 'beamsearch':
        decoderType = DecoderType.BeamSearch
    elif decoder == 'wordbeamsearch':
        decoderType = DecoderType.WordBeamSearch

    # train or validate on IAM dataset
    if train_b or validate:
        # load training data, create TF model
        loader = DataLoaderIAM(data_dir, batch_size,imgSize, maxTextLen,
                               fast)

        # save characters of model for inference mode
        open(FilePaths.fnCharList, 'w').write(str().join(loader.charList))

        # save words contained in dataset into file
        open(FilePaths.fnCorpus, 'w').write(
            str(' ').join(loader.trainWords + loader.validationWords))

        # execute training or validation
        if train_b:
            model = Model(loader.charList, decoderType)
            train(model, loader)
        elif validate:
            model = Model(loader.charList, decoderType, mustRestore=True)
            validate(model, loader)

    # infer text on test image
    else:
        model = Model(open(FilePaths.fnCharList).read(), decoderType, mustRestore=True,
                      dump=dump)
        infer(model, FilePaths.fnInfer)



    # test with img
    # decoderType = DecoderType.BestPath
    # path = r"F:\Studia\pythonProject\TestHDR\SimpleHTR\data\img\a04\a04-010\a04-010-00.png"
    # toread = cv2.imread(path)
    # model = Model(open(FilePaths.fnCharList).read(), decoderType, mustRestore=True)
    # infer(model, path)
    #
    # cv2.imshow("To read", toread)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
main()

/content/lines.txt
(None, 512, 128)
(None, 512, 256)
(None, 512, 128)
after dense1: (None, 512, 128)
concat: (None, 512, 1, 128)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Python: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
Tensorflow: 2.5.0
Init with new values
Epoch: 1
Train NN
Epoch: 1 Batch: 1/99 Loss: 448.2210998535156
Epoch: 1 Batch: 2/99 Loss: 444.1983337402344
Epoch: 1 Batch: 3/99 Loss: 436.6871337890625
Epoch: 1 Batch: 4/99 Loss: 429.7310791015625
Epoch: 1 Batch: 5/99 Loss: 413.6391296386719
Epoch: 1 Batch: 6/99 Loss: 386.3505554199219
Epoch: 1 Batch: 7/99 Loss: 319.3999938964844
Epoch: 1 Batch: 8/99 Loss: 221.37728881835938
Epoch: 1 Batch: 9/99 Loss: 193.74200439453125
Epoch: 1 Batch: 10/99 Loss: 190.7745819091797
Epoch: 1 Batch: 11/99 Loss: 186.76446533203125
Epoch: 1 Batch: 12/99 Loss: 178.27381896972656
Epoch: 1 Batch: 13/99 Loss: 170.50863647460938
Epoch: 1 Batch: 14/99 Loss: 173.119903564